In [39]:
# For Colab: Install FARM
!pip install torch==1.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install farm==0.5.0

Looking in links: https://download.pytorch.org/whl/torch_stable.html


# QA Fine-tuning

- Pretrained Model: [KoELECTRA](https://github.com/monologg/KoELECTRA)
- Dataset: [AIhub QA dataset](https://aihub.or.kr/aidata/86)
<center><img src="https://drive.google.com/uc?id=1RI9MoUB8QEfOzrvlVfOYaRvUeuNEW1cU" alt="Fine-tuning" width="100%" height="30%"></center>
- MLFlow: [link](https://public-mlflow.deepset.ai/#/experiments/314/runs/2791c81d303f4e8daeb8fd08ba4e4fca)
- Training environment: TITAN RTX x 4, batch_size = 96, early_stopped after 5200 batches.
<center><img src="https://drive.google.com/uc?id=1MALTUUDly-G0izbHCsdC_kXnoRv78rxg" alt="Fine-tuning" width="60%" height="30%"></center>
- Training results
<center><img src="https://drive.google.com/uc?id=1ogM3bEHifxm0qiJ_kw18_GOQKOVzCZG7" alt="Fine-tuning" width="100%" height="30%"></center>

In [ ]:
# if you not trained the model please use it
from pathlib import Path
import gdown
url = "https://drive.google.com/uc?id=1crJKVxSi7z9abx6xL-Vvxw-RUJShTJ3z"
ckpt_path = Path(".") / "ckpt"
if not ckpt_path.exists():
    ckpt_path.mkdir()
gdown.download(url, str(ckpt_path / "best_aihub.tar"), quiet=False)
!tar -xvf ./ckpt/best_aihub.tar -C ./ckpt

Downloading...
From: https://drive.google.com/uc?id=1crJKVxSi7z9abx6xL-Vvxw-RUJShTJ3z
To: /content/ckpt/best_aihub.tar
450MB [00:02, 210MB/s]


./best_aihub/
./best_aihub/processor_config.json
./best_aihub/language_model.bin
./best_aihub/prediction_head_0.bin
./best_aihub/vocab.txt
./best_aihub/language_model_config.json
./best_aihub/tokenizer_config.json
./best_aihub/special_tokens_map.json
./best_aihub/prediction_head_0_config.json


In [104]:
from farm.infer import Inferencer, QAInferencer
from pprint import PrettyPrinter

context1 = "제788회 로또 당첨번호‘2·10·11·19·35·39’ 보너스 번호 ‘29’ ... 1등 13명 각 14억원지난 6일 제788회 나눔로또 로또복권 추첨 결과 1등 당첨번호는 2·10·11·19·35·39의 6개다. 2등 보너스번호는 ‘29’이다. 로또 788회 로또당첨번호 6개를 모두 맞힌 1등 당첨자는 총 13명으로 각각 14억147만5154원씩 받게된다. 당첨번호 5개와 보너스 번호가 일치한 2등은 70명으로 4337만8993원, 당첨번호 5개를 맞힌 3등은 2257명으로 134만5384원 씩 받는다. 로또당첨번호 4개를 맞힌 4등(고정 당첨금 5만원)은 11만2504명, 로또 당첨번호 788회 3개가 일치한 5등(고정 당첨금 5000원)은 182만8701명이다. [사진=나눔로또 캡처]"
questions1 = [
    "3등은 당첨번호는 몇 개가 맞아야해?",
    "제788회 로또 당첨번호의 보너스 번호는 뭐야?"
]
answers1 = [
    {'text': '당첨번호 5개', 'answer_start': 203},
    {'text': ' ‘29’', 'answer_start': 38}
]

context2 = "동원FB의 펫푸드 전문 브랜드 뉴트리플랜이 국내 최초 참치알을 넣어 만든 애묘 습식 파우치 뉴트리플랜 모이스트루 4종(사진)을 출시했다고 21일 밝혔다. 동원FB는 최근 약 30억 원을 투자해 국내 창원공장에 펫푸드 생산을 위한 라인을 증설했다. 증설된 라인에서는 참치와 펫푸드 노하우를 활용한 다양한 애묘, 애견용 펫푸드를 선보일 예정이다. 그 첫 번째 시리즈가 국내 최초로 참치알을 활용해 만든 애묘용 습식파우치 모이스트루 4종(참치, 참치와 멸치, 참치와 닭고기, 참치와 연어)이다. 뉴트리플랜 모이스트루 4종은 육식동물 고양이의 건강한 습식 습관을 위한 고급 파우치 제품이다. 필수 아미노산인 타우린과 아르기닌을 다량 함유하고 있는 참치 붉은 살과 단백질, 오메가-3 지방산 및 미네랄이 풍부하며 기호성이 높은 참치알을 담았다. 또한 고양이의 하부요로기 질환에 도움을 주는 크랜베리와 장관환경과 배변상태 개선을 지원하는 프리바이오틱 성분인 이눌린을 첨가했다. 특히 28년 간 펫푸드를 만들어 일본에 수출해 온 동원FB가 국내에서 직접 만든 제품으로 믿을 수 있다. win5858@fnnews.com 김성원 기자"

questions2 = [
    "참치알을 넣어 만든 펫푸드를 출시한 곳은 어디지?",
    "동원FB가 새로 증설된 펫푸드 생산라인에서 선보인 첫 번째 시리즈는 뭐야?"
]
answers2 = [
    {'text': '동원FB', 'answer_start': 513},
    {'text': '뉴트리플랜 모이스트루 4종은 육식동물 고양이의 건강한 습식 습관을 위한 고급 파우치 제품이다', 'answer_start': 274}
]
basic_texts = [
    {
        "questions": questions1,
        "text": context1
    },
    {
        "questions": questions2,
        "text": context2
    }
]

In [99]:
infer_model = QAInferencer.load(    
    model_name_or_path="./ckpt/best_aihub",
    extraction_strategy="per_token",
    batch_size=2,
    max_seq_len=512, 
    doc_stride=128,
    task_type="question_answering",
)
result = infer_model.inference_from_dicts(dicts=basic_texts)
print()

03/31/2021 08:35:02 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None
03/31/2021 08:35:07 - INFO - farm.modeling.adaptive_model -   Found files for loading 1 prediction heads
03/31/2021 08:35:07 - WARNING - farm.modeling.prediction_head -   Some unused parameters are passed to the QuestionAnsweringHead. Might not be a problem. Params: {"training": false, "num_labels": 2, "ph_output_type": "per_token_squad", "model_type": "span_classification", "label_tensor_name": "question_answering_label_ids", "label_list": ["start_token", "end_token"], "metric": "squad", "name": "QuestionAnsweringHead"}
03/31/2021 08:35:07 - INFO - farm.modeling.prediction_head -   Prediction head initialized with size [768, 2]
03/31/2021 08:35:07 - INFO - farm.modeling.prediction_head -   Loading prediction head from ckpt/best_aihub/prediction_head_0.bin
03/31/2021 08:35:07 - INFO - farm.modeling.tokenization -   Loading tokenizer of type 'ElectraTok

In [106]:
import termcolor

def sprint(s, n=85):
    for i in range(len(s) // n):
        print("  ", s[i*n:(i*n+n)])
    print("  ", s[(i*n+n):])

true_answers = answers1 + answers2
contexts = [context1, context1, context2, context2]
for i, (batch) in enumerate(result):
    predictions = batch["predictions"][0]
    answers = predictions["answers"][0]
    
    context = contexts[i]
    true_answer = true_answers[i]

    question = predictions["question"]
    
    offset_answer_start = answers["offset_answer_start"]
    offset_answer_end = answers["offset_answer_end"]
    predict_answer = context[offset_answer_start:offset_answer_end]
    predict_answer_colored = termcolor.colored(predict_answer, "red", attrs=["bold"])
    predict_context_colored = context[:offset_answer_start] + predict_answer_colored + context[offset_answer_end:]
    print(termcolor.colored(f"[Question] {question}", attrs=["bold"]))
    print(f"[Predict: Context]")
    sprint(predict_context_colored)
    print(f"[Predict: Answer] {predict_answer} | {offset_answer_start}:{offset_answer_end}")
    
    true_answer_text = true_answer['text']
    true_answer_start = true_answer['answer_start']
    true_answer_end = true_answer_start+len(true_answer_text)
    true_answer_colored = termcolor.colored(true_answer_text, "blue", attrs=["bold"])
    true_context_colored = context[:true_answer_start] + true_answer_colored + context[true_answer_end:]
    print(f"[True: Context]")
    sprint(true_context_colored)
    print(f"[True: Answer] {true_answer_text} | {true_answer_start}:{true_answer_end}")
    print()

[Question] 3등은 당첨번호는 몇 개가 맞아야해?
[Predict: Context]
   제788회 로또 당첨번호‘2·10·11·19·35·39’ 보너스 번호 ‘29’ ... 1등 13명 각 14억원지난 6일 제788회 나눔로또 로또복권 추첨
    결과 1등 당첨번호는 2·10·11·19·35·39의 6개다. 2등 보너스번호는 ‘29’이다. 로또 788회 로또당첨번호 6개를 모두 맞힌 1등 당첨자
   는 총 13명으로 각각 14억147만5154원씩 받게된다. 당첨번호 5개와 보너스 번호가 일치한 2등은 70명으로 4337만8993원, 당첨번호 
   5개를 맞힌 3등은 2257명으로 134만5384원 씩 받는다. 로또당첨번호 4개를 맞힌 4등(고정 당첨금 5만원)은 11만2504명, 
   로또 당첨번호 788회 3개가 일치한 5등(고정 당첨금 5000원)은 182만8701명이다. [사진=나눔로또 캡처]
[Predict: Answer] 5개 | 251:253
[True: Context]
   제788회 로또 당첨번호‘2·10·11·19·35·39’ 보너스 번호 ‘29’ ... 1등 13명 각 14억원지난 6일 제788회 나눔로또 로또복권 추첨
    결과 1등 당첨번호는 2·10·11·19·35·39의 6개다. 2등 보너스번호는 ‘29’이다. 로또 788회 로또당첨번호 6개를 모두 맞힌 1등 당첨자
   는 총 13명으로 각각 14억147만5154원씩 받게된다. 당첨번호 5개와 보너스 번호가 일치한 2등은 70명으로 4337만899
   3원, 당첨번호 5개를 맞힌 3등은 2257명으로 134만5384원 씩 받는다. 로또당첨번호 4개를 맞힌 4등(고정 당첨금 5만원)은 11만2504명, 
   로또 당첨번호 788회 3개가 일치한 5등(고정 당첨금 5000원)은 182만8701명이다. [사진=나눔로또 캡처]
[True: Answer] 당첨번호 5개 | 203:210

[Question] 제788회 로또 당첨번호의 보너스 번호는 뭐야?
[Predict